In [2]:
import pickle
import numpy as np

In [3]:
file = open('pickle/CombineModel.pkl', 'rb')
model = pickle.load(file)
file.close()

file = open('pickle/vectorizer.pkl', 'rb')
vectorizer = pickle.load(file)
file.close

<function BufferedReader.close>

In [4]:
from packages.text import cleaning, flat
inputs = 'genshin is a shit game but i love you @obama'
cleaning = cleaning(inputs)
vec_inputs = vectorizer.transform([cleaning])

In [5]:
LRpred = model['LRmodel'].predict(vec_inputs)
SVCpred = model['SVCmodel'].predict(vec_inputs)
BNBpred = model['BNBmodel'].predict(vec_inputs)

LRpred_conf = max(flat(model['LRmodel'].predict_proba(vec_inputs)))
SVCpred_conf = max(flat(model['SVCmodel'].predict_proba(vec_inputs)))
BNBpred_conf = max(flat(model['BNBmodel'].predict_proba(vec_inputs)))

result = np.concatenate((LRpred, SVCpred, BNBpred))
result_conf = [LRpred_conf, SVCpred_conf, BNBpred_conf]


d = {
    'model': ['LRpred', 'SVCpred', 'BNBpred'],
    'predict': result,
    'confidence': result_conf
}
x = pd.DataFrame(d)
x

<IPython.core.display.Javascript object>

,model,predict,confidence
0,LRpred,POSITIVE,0.516129
1,SVCpred,NEGATIVE,0.697854
2,BNBpred,POSITIVE,0.622460


In [6]:
result_pred = x.predict.mode()[0]
confidence = round(x[x['predict'] == x.predict.mode()[0]]['confidence'].max()*100,2)
print(f"Sentiment: {result_pred}")
print(f"Confidence Level: {confidence}%")

Sentiment: POSITIVE
Confidence Level: 62.25%
